In [1]:

%pip install pyserini==0.12.0
%pip install glove-python-binary
%pip install glove

import numpy as np
import json
import re
from urllib.request import urlopen
import nltk
from pyserini import search
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

np.random.seed(1)


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached glove-1.0.2.tar.gz (44 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-12-arm64-cpython-310
      creating build/lib.macosx-12-arm64-cpython-310/glove
      copying glove/__ini

[nltk_data] Downloading package wordnet to /Users/hstack/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/hstack/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hstack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
docids = set()
# queryId -> docId -> relevanceScore
qrels = dict()
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  new_docid = docid.decode('UTF-8')
  docids.add(new_docid)

  if int(qid) in qrels:
    qrels[int(qid)][new_docid] = int(score)
  else:
    qrels[int(qid)] = dict()
    qrels[int(qid)][new_docid] = int(score)


docids = list(docids)
np.random.shuffle(docids)
print(len(docids), docids[0])
print(len(qrels))

174787 FT923-3828
249


In [3]:
data = []
searcher = search.SimpleSearcher.from_prebuilt_index('robust04')
for docid in docids[:50]:
  try:
    text = searcher.doc(docid).raw()
    data.append((docid, text))
  except:
    continue

cleaned_data = []
for docid, text in data:
  new_text = text.replace('<P>', ' ')
  new_text = new_text.replace('</P>', ' ')
  new_text = new_text.replace('<DATE>', ' ')
  new_text = new_text.replace('</DATE>', ' ')
  new_text = new_text.replace('<HEADLINE>', ' ')
  new_text = new_text.replace('</HEADLINE>', ' ')
  new_text = new_text.replace('<TEXT>', ' ')
  new_text = new_text.replace('</TEXT>', ' ')
  new_text = new_text.replace('\n', ' ')
  new_text = new_text.replace("\'", '')
  new_text = new_text.replace('--', ' ')
  cleaned_data.append((docid, new_text))
  
cleaned_data[0]

Attempting to initialize pre-built index robust04.
/Users/hstack/.cache/pyserini/indexes/index-robust04-20191213.15f3d001489c97849a010b0a4734d018 already exists, skipping download.
Initializing robust04...


('FT923-3828',
 ' 920910     FT  10 SEP 92 / Technology: Lifes little ups and downs - Escalators may have a humdrum role but building the longest in western Europe has proved to be no easy task     The diary of Richard Burbage, manager of Harrods in 1898, recalls that shoppers overcome by the experience of travelling on Britains first escalator were revived with brandy or sal volatile dispensed by an attendant positioned at the top. Burbage, according to the Shell Book of Firsts, had an unreasoning dislike of lifts, and persuaded the Knightsbridge store to instal the 40ft-long Reno Inclined Elevator, patented by Jesse W. Reno of New York just six years earlier. Nearly a century later, would-be passengers on the longest escalators in western Europe, at the new Angel station on the London Underground, might feel they deserve similar resuscitation after the problems of the past month. Dogged by difficulties, the three 60m-long escalators are still not functioning properly, delaying the op

In [4]:
stemmer = nltk.stem.PorterStemmer()
stop_words = set(nltk.corpus.stopwords.words('english'))

def tokenize(doc_id_and_text):
    text = doc_id_and_text[1]
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    tokens = pattern.findall(text)
    stemmed_tokens = []
    for token in tokens:
      stemmed_token = stemmer.stem(token.replace("'", ''))
      if stemmed_token not in stop_words:
        stemmed_tokens.append(stemmed_token)
    return (doc_id_and_text[0],stemmed_tokens)

tokens_by_doc = [tokenize(doc) for doc in cleaned_data[:25]]
print(tokens_by_doc[0])

doc_id_to_tokens = dict()
for tokens in tokens_by_doc:
  doc_id_to_tokens[tokens[0]] = tokens[1]

('FT923-3828', ['ft', 'sep', 'technolog', 'life', 'littl', 'escal', 'may', 'humdrum', 'role', 'build', 'longest', 'western', 'europ', 'ha', 'prove', 'easi', 'task', 'diari', 'richard', 'burbag', 'manag', 'harrod', 'recal', 'shopper', 'overcom', 'experi', 'travel', 'britain', 'first', 'escal', 'reviv', 'brandi', 'sal', 'volatil', 'dispens', 'attend', 'posit', 'top', 'burbag', 'accord', 'shell', 'book', 'first', 'unreason', 'dislik', 'lift', 'persuad', 'knightsbridg', 'store', 'instal', '40ft', 'long', 'reno', 'inclin', 'elev', 'patent', 'jess', 'w', 'reno', 'new', 'york', 'six', 'year', 'earlier', 'nearli', 'centuri', 'later', 'would', 'passeng', 'longest', 'escal', 'western', 'europ', 'new', 'angel', 'station', 'london', 'underground', 'might', 'feel', 'deserv', 'similar', 'resuscit', 'problem', 'past', 'month', 'dog', 'difficulti', 'three', '60m', 'long', 'escal', 'still', 'function', 'properli', 'delay', 'open', 'long', 'await', 'pound', '70m', 'new', 'station', 'islington', 'commut'

In [5]:
# Use the topics from the homework to search by document
from pyserini.search import get_topics
topics = get_topics('robust04')
print(len(topics))

idx = 0
for topic in topics:
    if idx < 5:
        print(topic)
        print(topics[topic]['title'])
    else:
        break

250
350
Health and Computer Terminals
351
Falkland petroleum exploration
352
British Chunnel impact
353
Antarctica exploration
354
journalist risks
355
ocean remote sensing
356
postmenopausal estrogen Britain
357
territorial waters dispute
358
blood-alcohol fatalities
359
mutual fund predictors
360
drug legalization benefits
361
clothing sweatshops
362
human smuggling
363
transportation tunnel disasters
364
rabies
365
El Nino
366
commercial cyanide uses
367
piracy
368
in vitro fertilization
369
anorexia nervosa bulimia
370
food/drug laws
371
health insurance holistic
372
Native American casino
373
encryption equipment export
374
Nobel prize winners
375
hydrogen energy
376
World Court
377
cigar smoking
378
euro opposition
379
mainstreaming
380
obesity medical treatment
381
alternative medicine
382
hydrogen fuel automobiles
383
mental illness drugs
384
space station moon
385
hybrid fuel cars
386
teaching disabled children
387
radioactive waste
388
organic soil enhancement
389
illegal tec

In [10]:
import json

with open('embeddings.json') as f:
    word_embeddings = json.load(f)
print(word_embeddings['octob'])

[0.8538881190934323, 0.3578883330164455, -0.1872723552896967, -0.3523864200200205, -0.46351744368155007, 0.6468629776968987, 0.6614826316673852, -0.3668020283483838, 0.9602728314242588, 0.3856306968482955]


In [9]:
glove_embeddings = dict()
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove_embeddings[word] = vector

In [11]:
# We need to get the embeddings for every word in each document document (you do np.mean() on the embeddings list to do this)
import numpy as np
import nltk

VECTOR_SIZE = 10
GLOVE_VECTOR_SIZE = 50
stemmer = nltk.stem.PorterStemmer()

def get_document_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens) < 1:
        return np.zeros(VECTOR_SIZE)
    else:
        for token in doc_tokens:
            stemmed_token = stemmer.stem(token)
            if stemmed_token in word_embeddings:
                embeddings.append(word_embeddings[stemmed_token])
            else:
                embeddings.append(np.random.normal(0, 1, VECTOR_SIZE))
        # average the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

def get_glove_document_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens) < 1:
        return np.zeros(GLOVE_VECTOR_SIZE)
    else:
        for token in doc_tokens:
            stemmed_token = stemmer.stem(token)
            if stemmed_token in glove_embeddings:
                embeddings.append(glove_embeddings[stemmed_token])
            else:
                embeddings.append(np.random.normal(0, 1, GLOVE_VECTOR_SIZE))
        # average the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

document_embeddings = dict()
for doc in tokens_by_doc:
    document_embeddings[doc[0]] = get_document_embedding(doc[1])

glove_document_embeddings = dict()
for doc in tokens_by_doc:
    glove_document_embeddings[doc[0]] = get_glove_document_embedding(doc[1])

In [12]:
# When given a query, get its embedding and then do cosine similarity with each of the embedding of the documents
import heapq

def cosim(v1, v2):
  return np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2))

def get_our_top_hundred_documents(query):
    # Maybe do something different for query tokens 
    query_tokens = query.split(' ')
    query_embedding = get_document_embedding(query_tokens)
    
    similarities = []

    idx = 0
    for document in document_embeddings:
      # Will prob have to change this
      document_name = document
      document_embedding = document_embeddings[document]
      similarity = cosim(document_embedding, query_embedding)
      if idx < 100:
        heapq.heappush(similarities, (similarity, document_name))
      else:
        heapq.heapreplace(similarities, (similarity, document_name))
      idx += 1
    return heapq.nlargest(100, similarities)

def get_glove_top_hundred_documents(query):
  # Maybe do something different for query tokens 
    query_tokens = query.split(' ')
    query_embedding = get_glove_document_embedding(query_tokens)
    
    similarities = []

    idx = 0
    for document in glove_document_embeddings:
      # Will prob have to change this
      document_name = document
      document_embedding = glove_document_embeddings[document]
      similarity = cosim(document_embedding, query_embedding)
      if idx < 100:
        heapq.heappush(similarities, (similarity, document_name))
      else:
        heapq.heapreplace(similarities, (similarity, document_name))
      idx += 1
    return heapq.nlargest(100, similarities)

def get_top_hundred_fast_text_documents(query):
  hits = searcher.search(query, 100)
  return [(hit.score, hit.docid) for hit in hits]


def get_top_hundred_ctrl_plus_f_documents(query):
  relevant_docs = set()

  query_tokens = []
  for token in query.split(' '):
    query_tokens.append(stemmer.stem(token))
  for tup in doc_id_to_tokens:
    doc_id = tup[0]
    tokens = tup[1]
    last_token_index = len(tokens) - 1
    current_token_index = 0
    occurences = 0

    for token in tokens:

      if token == query_tokens[current_token_index]:
        if current_token_index == last_token_index:
          occurences += 1
          current_token_index = 0
          
        else:
          current_token_index += 1
    if occurences > 0:
      relevant_docs.add((doc_id, occurences))
    doc_list = list(relevant_docs)
    doc_list.sort(key=lambda x: x[1], reverse=True)
    return doc_list
        



In [15]:
# topic is a qid -> dict['title] is the query
first_query_ranking = get_our_top_hundred_documents(topics[301]['title'])
for i in range(10):
    print(first_query_ranking[i])
print('------')


(0.9426818372048587, 'LA100689-0017')
(0.9202320420408827, 'LA060389-0042')
(0.919560374919724, 'FT932-4890')
(0.9142895750234512, 'FBIS3-27512')
(0.907285404929815, 'FBIS3-2095')
(0.8861437333938965, 'FT923-3828')
(0.8838506893854416, 'FT923-4849')
(0.8816781025110876, 'FT933-4868')
(0.8763627585450621, 'FT944-3280')
(0.8749924285238361, 'LA102489-0132')
------


In [16]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

In [17]:
def get_relevant_docs(query_id):
    relevant_docs = set()
    if query_id in qrels:
        for document_id in qrels[query_id]:
            if document_id in qrels[query_id]:
                if qrels[query_id][document_id] == 1:
                    relevant_docs.add(document_id)
    return relevant_docs

# Calculate the MAP@100 for our model, CTRL + F, fasttext, and GloVe
def map_hundred(given_topics, model_type):
    average_precision_values = []
    for topic in given_topics:
        query = topics[topic]['title']
        actual_relevant_docs = get_relevant_docs(topic)
        if model_type == "ours":
            our_relevant_docs = get_our_top_hundred_documents(query)
        elif model_type == "fasttext":
            our_relevant_docs = get_top_hundred_fast_text_documents(query)
        elif model_type == 'ctrl-f':
            our_relevant_docs = get_top_hundred_ctrl_plus_f_documents(query)
        elif model_type == 'glove':
            our_relevant_docs = get_glove_top_hundred_documents(query)
        
        summed_precision = 0
        relevant_count = 0
        total_count = 0
        for i in range(len(our_relevant_docs)):
            should_add = False
            if our_relevant_docs[i][1] in actual_relevant_docs:
                relevant_count += 1
                should_add = True
            total_count += 1
            if should_add:
                summed_precision += (relevant_count / total_count)
        if relevant_count > 0:
            average_precision_values.append(summed_precision / relevant_count)
    average_sum = sum(average_precision_values)
    if len(average_precision_values) > 0:
        return average_sum / len(average_precision_values)
    return 0


In [18]:
our_map_at_hundred = map_hundred(test_topics, 'ours')
print(our_map_at_hundred)
fasttext_map_at_hundred = map_hundred(test_topics, 'fasttext')
print(fasttext_map_at_hundred)
ctrl_f_map_at_hundred = map_hundred(test_topics, 'ctrl-f')
print(ctrl_f_map_at_hundred)
glove_map_at_100 = map_hundred(test_topics, 'glove')
print(glove_map_at_100)

0.2361111111111111
0.4133287942842544
0
1.0


In [124]:
# relevant docs for 350, 351, 352
query_one = topics[350]['title']
query_two = topics[351]['title']
query_three = topics[352]['title']

our_top_3_one = get_our_top_hundred_documents(query_one)[:3]
fast_text_top_3_one = get_top_hundred_fast_text_documents(query_one)[:3]
ctrl_f_top_3_one = get_top_hundred_ctrl_plus_f_documents(query_one)[:3]
glove_top_3_one = get_glove_top_hundred_documents(query_one)[:3]

our_top_3_two = get_our_top_hundred_documents(query_two)[:3]
fast_text_top_3_two = get_top_hundred_fast_text_documents(query_two)[:3]
ctrl_f_top_3_two = get_top_hundred_ctrl_plus_f_documents(query_two)[:3]
glove_top_3_two = get_glove_top_hundred_documents(query_two)[:3]

our_top_3_three = get_our_top_hundred_documents(query_three)[:3]
fast_text_top_3_three = get_top_hundred_fast_text_documents(query_three)[:3]
ctrl_f_top_3_three = get_top_hundred_ctrl_plus_f_documents(query_three)[:3]
glove_top_3_three = get_glove_top_hundred_documents(query_three)[:3]

print(our_top_3_one)
print(fast_text_top_3_one)
print(ctrl_f_top_3_one)
print(glove_top_3_one)
print("----------")
print(our_top_3_two)
print(fast_text_top_3_two)
print(ctrl_f_top_3_two)
print(glove_top_3_two)
print("----------")
print(our_top_3_three)
print(fast_text_top_3_three)
print(ctrl_f_top_3_three)
print(glove_top_3_three)
# relevant_docs = []
# for doc_id in qrels[350]:
#     if qrels[350][doc_id] == 1:
#         relevant_docs.append(doc_id)
# print(relevant_docs)
# print("---------")
# relevant_docs = []
# for doc_id in qrels[351]:
#     if qrels[351][doc_id] == 1:
#         relevant_docs.append(doc_id)
# print(relevant_docs)
# print("---------")
# relevant_docs = []
# for doc_id in qrels[352]:
#     if qrels[352][doc_id] == 1:
#         relevant_docs.append(doc_id)
# print(relevant_docs)
# print("---------")

[(0.24606010343897608, 'FBIS4-20451'), (0.2010871874979118, 'FT934-8936'), (0.19497096592973345, 'FT943-2121')]
[(8.458999633789062, 'LA052290-0188'), (7.700099945068359, 'LA060690-0112'), (7.6209001541137695, 'FT922-6787')]
[]
----------
[(0.4478520768234519, 'LA070390-0002'), (0.3396618777163815, 'FT941-2885'), (0.2303453807556575, 'FT934-2731')]
[(10.147199630737305, 'FT941-9999'), (9.906599998474121, 'FT934-4848'), (9.766200065612793, 'FT922-15099')]
[]
----------
[(0.6961953241735617, 'LA100689-0099'), (0.626313290366125, 'FT934-16490'), (0.6255856252274765, 'FT934-2731')]
[(8.918700218200684, 'FT934-11803'), (8.74779987335205, 'LA120290-0163'), (8.687600135803223, 'FT934-10925')]
[]
